In [1]:
import os
import datetime as dt
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
import json
import keras

BASE_SIZE = 256
NCATS = 340

Using TensorFlow backend.


In [2]:
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [3]:
def image_generator_xd(size, batchsize, lw=2, time_color=True, proprocess = None):
    while True:
        for df in pd.read_csv('../input/train_all.csv', chunksize=batchsize):
            df['drawing'] = df['drawing'].apply(json.loads)
            x = np.zeros((len(df), size, size, 1))
            for i, raw_strokes in enumerate(df.drawing.values):
                x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                         time_color=time_color)
            if proprocess is not None:
                x = preprocess_input(x).astype(np.float32)
            y = keras.utils.to_categorical(df.word, num_classes=NCATS)
            yield x, y

In [5]:
datagen = image_generator_xd(64, 16)

In [40]:
x,y = next(datagen)

In [ ]:
def df_to_image_array_xd(df, size, lw=2, time_color=True):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    if proprocess is not None:
        x = preprocess_input(x).astype(np.float32)
    return x